# Code for training the holistic model

## Import Libraries 


In [1]:
import tensorflow as tf
import numpy as np
import os
from datetime import datetime

from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.callbacks import TensorBoard
from keras.callbacks import ModelCheckpoint

from sklearn.model_selection import train_test_split
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, classification_report

## Setup Variables

In [2]:
FOLDER_NAME = 'dataset'
ALL_CLASSES = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']

In [3]:
# Create label map, representing each class as a number
label_map = {}
for (root, folders, files) in os.walk(FOLDER_NAME):
    for foldername in folders:
        if foldername in ALL_CLASSES:
            label_map[foldername] = ALL_CLASSES.index(foldername)

print(label_map)

{'a': 0, 'b': 1, 'c': 2, 'd': 3, 'e': 4, 'f': 5, 'g': 6, 'h': 7, 'i': 8, 'j': 9, 'k': 10, 'l': 11, 'm': 12, 'n': 13, 'o': 14, 'p': 15, 'q': 16, 'r': 17, 's': 18, 't': 19, 'u': 20, 'v': 21, 'w': 22, 'x': 23, 'y': 24, 'z': 25}


In [10]:
# Get all datset data with its label and put it in a list
sequence, label = [], []
target_length = 14
for (root, folders, files) in os.walk(FOLDER_NAME):
    total_file = 0
    for filename in files:
        file_path = os.path.join(os.path.relpath(
            root, FOLDER_NAME), filename)
        if (filename.endswith('.npy') and os.path.split(file_path)[0] in ALL_CLASSES):
            res = np.load(f'{FOLDER_NAME}/{file_path}')
            for _ in range(target_length-res.shape[0]):
                res = np.vstack((res, res[-1, :]))
            res = res[:, -126:]
            sequence.append(np.array(res))
            label.append(label_map[os.path.basename(root[-1])])
            total_file += 1
    print(f"Total files: {total_file} --- {root}")

print(np.array(sequence).shape)
print(np.array(label).shape)

Total files: 0 --- dataset
Total files: 60 --- dataset\a
Total files: 40 --- dataset\b
Total files: 60 --- dataset\c
Total files: 40 --- dataset\d
Total files: 35 --- dataset\e
Total files: 30 --- dataset\f
Total files: 30 --- dataset\g
Total files: 30 --- dataset\h
Total files: 30 --- dataset\i
Total files: 40 --- dataset\j
Total files: 31 --- dataset\k
Total files: 40 --- dataset\l
Total files: 50 --- dataset\m
Total files: 30 --- dataset\n
Total files: 41 --- dataset\o
Total files: 35 --- dataset\p
Total files: 40 --- dataset\q
Total files: 32 --- dataset\r
Total files: 45 --- dataset\s
Total files: 36 --- dataset\t
Total files: 29 --- dataset\u
Total files: 35 --- dataset\v
Total files: 30 --- dataset\w
Total files: 40 --- dataset\x
Total files: 30 --- dataset\y
Total files: 60 --- dataset\z
(999, 14, 126)
(999,)


In [11]:
# Dataset duplication if necessary
n = 2
sequence = np.concatenate([sequence] * n, axis=0)
label = np.concatenate([label] * n, axis=0)


print(np.array(sequence).shape)
print(np.array(label).shape)

(1998, 14, 126)
(1998,)


In [12]:
tf.config.list_physical_devices('GPU')


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

## Training Data

In [13]:
X_train, X_test, y_train, y_test = train_test_split(np.array(sequence), tf.keras.utils.to_categorical(
    np.array(label).astype(int), num_classes=np.array(ALL_CLASSES).shape[0], dtype='float32'), test_size=0.2)

print(X_train.shape, X_test.shape)


(1598, 14, 126) (400, 14, 126)


In [14]:
current_time = datetime.now().strftime("%Y%m%d-%H%M%S")

log_dir = os.path.join('Logs', current_time)
tb_callback = TensorBoard(log_dir=log_dir)

print(log_dir)

model = Sequential()
model.add(LSTM(64, return_sequences=True,
          activation='tanh', input_shape=(14, 126)))
model.add(LSTM(128, return_sequences=True, activation='tanh'))
model.add(LSTM(64, return_sequences=False, activation='tanh'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(np.array(ALL_CLASSES).shape[0], activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy',
              metrics=['categorical_accuracy'])

Logs\20240628-163454


In [15]:
model.fit(X_train, y_train, epochs=100, callbacks=[
          tb_callback])

Epoch 1/100
50/50 [==============================] - 8s 14ms/step - loss: 3.0158 - categorical_accuracy: 0.1389
Epoch 2/100
50/50 [==============================] - 1s 13ms/step - loss: 2.4948 - categorical_accuracy: 0.2103
Epoch 3/100
50/50 [==============================] - 1s 13ms/step - loss: 2.1224 - categorical_accuracy: 0.2553
Epoch 4/100
50/50 [==============================] - 1s 12ms/step - loss: 1.8864 - categorical_accuracy: 0.3023
Epoch 5/100
50/50 [==============================] - 1s 12ms/step - loss: 1.8738 - categorical_accuracy: 0.2916
Epoch 6/100
50/50 [==============================] - 1s 12ms/step - loss: 1.6735 - categorical_accuracy: 0.3548
Epoch 7/100
50/50 [==============================] - 1s 13ms/step - loss: 1.6954 - categorical_accuracy: 0.3385
Epoch 8/100
50/50 [==============================] - 1s 13ms/step - loss: 1.5381 - categorical_accuracy: 0.3861
Epoch 9/100
50/50 [==============================] - 1s 21ms/step - loss: 1.4917 - categorical_accuracy:

In [16]:
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 14, 64)            48896     
                                                                 
 lstm_1 (LSTM)               (None, 14, 128)           98816     
                                                                 
 lstm_2 (LSTM)               (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 26)                858       
                                                                 
Total params: 204,218
Trainable params: 204,218
Non-trai

In [17]:
res = model.predict(X_test)

13/13 [==============================] - 1s 9ms/step


In [18]:


# # Assume X_test and y_test are your test dataset
# # Make predictions
# y_pred = model.predict(X_test)

# # Convert predictions to class labels
# y_pred_classes = np.argmax(y_pred, axis=1)
# y_true = np.argmax(y_test, axis=1)  # if y_test is one-hot encoded

# # Calculate accuracy
# accuracy = accuracy_score(y_true, y_pred_classes)

# # Calculate precision, recall, and F1 score for each class
# precision = precision_score(y_true, y_pred_classes, average='weighted')
# recall = recall_score(y_true, y_pred_classes, average='weighted')
# f1 = f1_score(y_true, y_pred_classes, average='weighted')

# # Print the results
# print(f"Accuracy: {accuracy}")
# print(f"Precision: {precision}")
# print(f"Recall: {recall}")
# print(f"F1 Score: {f1}")

# # Detailed classification report
# report = classification_report(y_true, y_pred_classes)
# print(report)


In [19]:
actions = np.array(ALL_CLASSES)
testing = 13
print(actions[np.argmax(res[testing])], actions[np.argmax(y_test[testing])])

i i


In [20]:
model.save(f'{log_dir}/action.h5')